In [72]:
from pyspark.sql import SparkSession
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.unischema import dict_to_spark_row
import numpy as np
from petastorm.unischema import Unischema, UnischemaField
from pyspark.sql.types import IntegerType,StringType,DoubleType
from petastorm.codecs import ScalarCodec

import pyarrow.dataset as ds
import os

In [61]:

spark = SparkSession.builder.appName("ParquetToDataset").getOrCreate()
print(f"spark: {spark}")
parquet_data = spark.read.parquet("/mnt/cephfs/tpch_sf2_parquet/customer")
print(f"parquet data type: {parquet_data.dtypes}")
schema = parquet_data.schema
print(f"parquet schema: {parquet_data.schema}")

spark: <pyspark.sql.session.SparkSession object at 0x7f739f5aba30>
parquet data type: [('c_custkey', 'bigint'), ('c_name', 'string'), ('c_address', 'string'), ('c_nationkey', 'bigint'), ('c_phone', 'string'), ('c_acctbal', 'double'), ('c_mktsegment', 'string'), ('c_comment', 'string')]
parquet schema: StructType([StructField('c_custkey', LongType(), True), StructField('c_name', StringType(), True), StructField('c_address', StringType(), True), StructField('c_nationkey', LongType(), True), StructField('c_phone', StringType(), True), StructField('c_acctbal', DoubleType(), True), StructField('c_mktsegment', StringType(), True), StructField('c_comment', StringType(), True)])


In [62]:
sc = spark.sparkContext
print(sc)

<SparkContext master=local[*] appName=ParquetToDataset>


In [63]:
spark1 = SparkSession.builder.config('spark.driver.memory', '2g').master('local[2]').getOrCreate()
sc1 = spark.sparkContext
print(f"sc1: {sc1}")

sc1: <SparkContext master=local[*] appName=ParquetToDataset>


In [64]:
def row_generator(x):
    """Returns a single entry in the generated dataset. Return a bunch of random values as an example."""
    return {'id': x,
            'image1': np.random.randint(0, 255, dtype=np.uint8, size=(128, 256, 3)),
            'array_4d': np.random.randint(0, 255, dtype=np.uint8, size=(4, 128, 30, 3))}

In [65]:
def row_generator(x):
    """Returns a single entry in the generated dataset. Return a bunch of random values as an example."""
    return {'id': x,
            'image1': np.random.randint(0, 255, dtype=np.uint8, size=(128, 256, 3)),
            'array_4d': np.random.randint(0, 255, dtype=np.uint8, size=(4, 128, 30, 3))}

In [66]:
CustomerSchema = Unischema('CustomerSchema', [
    UnischemaField('c_custkey', np.int32, (), ScalarCodec(IntegerType()), False),
    UnischemaField('c_name', np.str_, (), ScalarCodec(StringType()), False),
    UnischemaField('c_address', np.str_, (), ScalarCodec(StringType()), False),
    UnischemaField('c_nationkey', np.int32, (), ScalarCodec(IntegerType()), False),
    UnischemaField('c_phone', np.str_, (), ScalarCodec(StringType()), False),
    UnischemaField('c_acctbal', np.float64, (), ScalarCodec(DoubleType()), False),
    UnischemaField('c_mktsegment', np.str_, (), ScalarCodec(StringType()), False),
    UnischemaField('c_comment', np.str_, (), ScalarCodec(StringType()), False)
])

In [73]:
# Wrap dataset materialization portion. Will take care of setting up spark environment variables as
# well as save petastorm specific metadata
rows_count = 10
output_url = 'file:///mnt/cephfs/test'
rowgroup_size_mb = 256

# Read the Parquet file
df = spark.read.parquet('/mnt/cephfs/tpch_sf2_parquet/customer')
with materialize_dataset(spark, output_url, CustomerSchema):

    # rows_rdd = sc.parallelize(range(rows_count))\
    #     .map(row_generator)\
    #     .map(lambda x: dict_to_spark_row(schema, x))

    # spark.createDataFrame(rows_rdd, schema.as_spark_schema()) \
    df.write.mode('overwrite').parquet('file:///home/yue21/test')


/home/yue21/petastorm/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
/home/yue21/petastorm/petastorm/fs_utils.py:89: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem_factory = lambda: pyarrow.localfs


OSError: Passed non-file path: /mnt/cephfs/test

In [74]:
for _ in range(100):
    # dataset_path = str(sys.argv[1])
    dataset_path = "file:///home/yue21/test"
    # query_no = int(sys.argv[2])
    query_no = 22
    # format = str(sys.argv[3])
    format = ds.SkyhookFileFormat("parquet", "/etc/ceph/ceph.conf")

    data = list()
    lineitem = ds.dataset((dataset_path), format=format)
    # supplier = ds.dataset(os.path.join(dataset_path, "supplier"), format=format)
    # customer = ds.dataset(os.path.join(dataset_path, "customer"), format=format)